# 🍎 Chat Completions with AIProjectClient 🍏
In this notebook, we'll demonstrate how to perform **Chat Completions** using the **Azure AI Foundry SDK**. We will be performing the following steps:

1. **Initialize** an AIProjectClient.
2. **Obtain** a OpenAI client from Project client.
3. **Obtain** a Responses client from OpenAI client.
4. **Use** a **prompt template** to add system context.
5. **Send** user prompts in a health & fitness theme.

# 🏋️ Health-Fitness Disclaimer
> **This example is for demonstration only and does not provide real medical advice.** Always consult a professional for health or medical-related questions.

## Prerequisites
Before starting this notebook, please ensure you have completed all prerequisites listed in the root [README.md](../../README.md).

Let's get started! 🎉

<img src="seq-diagrams/1-chat.png" alt="Chat Completion Sequence Diagram" width="600"/>

# 1. Initial Setup

Load environment variables, create an AIProjectClient, and fetch a ResponsesClient. We'll also define a prompt template to show how you might structure a system message. Review [prompt engineering techniques](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/concepts/prompt-engineering?context=%2Fazure%2Fai-foundry%2Fcontext%2Fcontext) for additional insights and patter

In [ ]:
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System.IO;
using System.ClientModel.Primitives;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Azure.Core;
using OpenAI;
using OpenAI.Responses;
using dotenv.net;  

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");

TokenCredential credential;
AIProjectClient projectClient;

try
{
    var cliCredentials = new AzureCliCredential(new AzureCliCredentialOptions{ TenantId = tenantId});
    var interactiveCredentials = new InteractiveBrowserCredential(new InteractiveBrowserCredentialOptions{TenantId = tenantId});
    var chainedCredentials = new ChainedTokenCredential(cliCredentials, interactiveCredentials);
    
    projectClient = new AIProjectClient(new Uri(projectEndpoint), chainedCredentials);

    Console.WriteLine("✅ Successfully created AIProjectClient");
    
}
catch (Exception ex)
{
    Console.WriteLine("❌ Error initializing client: " + ex.Message);
}

# Prompt Template

We'll define a quick system message that sets the context as a friendly, disclaimer-providing fitness assistant. This will be passed as instructions to the chat completion client.

```txt
You are FitChat GPT, a helpful fitness assistant.
Always remind users: I'm not a medical professional.
Be friendly, provide general advice.
...
```

We'll then pass user content as a **user** message.

In [ ]:
#pragma warning disable OPENAI001

string ChatWithFitnessAssistant(string userInput)
{
    // Obtain Chat Completions client from AIProjectClient
    var responsesClient = projectClient.GetProjectOpenAIClient().GetProjectResponsesClientForModel(modelDeploymentName);

    // Define system prompt for health & fitness context
    string systemPrompt = @"You are FitChat GPT, a friendly fitness assistant. 
                            Always remind users: I'm not a medical professional.
                            Answer with empathy and disclaimers.";
    var request = new CreateResponseOptions()
        {
            InputItems = { (ResponseItem)ResponseItem.CreateSystemMessageItem(systemPrompt), (ResponseItem)ResponseItem.CreateUserMessageItem(userInput) }
        };

    var response = responsesClient.CreateResponse(request);
    // Return the assistant's reply
    return response.Value.GetOutputText();
}

# 2. Try Chat Completions 🎉

We'll call the function with a user question about health or fitness, and see the result. Feel free to modify the question or run multiple times!

In [ ]:
var userQuestion = "How can i start a beginner workout routine at home?";
var reply = ChatWithFitnessAssistant(userQuestion);
Console.WriteLine("🗣️ User: " + userQuestion);
Console.WriteLine("🤖 Assistant: " + reply);

# 3. Another Example: Prompt Template with Fill-Ins 📝

We can go a bit further and add placeholders in the system message. For instance, imagine we have a userName or goal. We'll show a minimal example.

In [ ]:
#pragma warning disable OPENAI001

string ChatWithTemplate(string userInput, string userName, string goal)
{
    // Obtain Chat Completions client from AIProjectClient
    var responsesClient = projectClient.GetProjectOpenAIClient().GetProjectResponsesClientForModel(modelDeploymentName);

    // Define system prompt for health & fitness context
    string systemPrompt = $@"You are FitChat GPT, an AI personal trainer for {userName}. 
                            Your user wants to achieve: {goal}.
                            Always remind users: I'm not a medical professional.
                            Remind them you're not a medical professional. Offer friendly advice.";

    var request = new CreateResponseOptions()
    {
        InputItems = { (ResponseItem)ResponseItem.CreateSystemMessageItem(systemPrompt), (ResponseItem)ResponseItem.CreateUserMessageItem(userInput) }
    };
    var response = responsesClient.CreateResponse(request);
    // Return the assistant's reply
    return response.Value.GetOutputText();
}

In [ ]:
userQuestion = "What kind of home exercise do you recommend for a busy schedule?";
reply = ChatWithTemplate(userQuestion, "Alex", "increase muscle tone & endurance");
Console.WriteLine("🗣️ User: " + userQuestion);
Console.WriteLine("🤖 Assistant: " + reply);

# 🎉 Congratulations!

You've successfully performed chat completions with the Azure AI Foundry's AIProjectClient and OpenAI Responses client. You've also seen how to incorporate prompt templates to tailor your system instructions.

Head to [2-embeddings.ipynb](2-embeddings.ipynb) for the next part of the workshop! 🎯